# Prep

In [ ]:
# Install dependencies
# %pip install -q -U ipywidgets transformers tqdm
# %pip install -q -U seqeval
# %pip install -q -U accelerate
# %pip install -q -U transformers[torch]
# %pip install -q --upgrade -U torch torchvision torchaudio torchtext
# %pip install -q dill==0.3.1.1
# %pip install -q numpy==1.14.3
# %pip install -q pyarrow==0.3.8
# %pip install -q multiprocess==0.70.16
# %pip install -q -U datasets==2.6.0
# %pip install fsspec==2023.9.2

In [3]:
import datasets
import torch
import torchtext
from datasets import load_dataset, Features, Value
import matplotlib.pyplot as plt
import nltk
from datasets import DatasetDict, Dataset
from sklearn.metrics import classification_report
from transformers import AutoTokenizer
import transformers
from transformers import AutoModelForTokenClassification, TrainingArguments, Trainer, pipeline, Pipeline
from transformers import DataCollatorForTokenClassification
from datasets import load_metric
import numpy as np
import dill
import gc
from transformers import Trainer
import torch.nn as nn
from datasets import DatasetDict, Dataset
from sklearn.pipeline import Pipeline

In [ ]:
SEED = 1234
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
torch.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

print("PyTorch Version: ", torch.__version__)
print("torchtext Version: ", torchtext.__version__)
print(f"Using {'GPU' if str(DEVICE) == 'cuda' else 'CPU'}.")

# Pre Processing

In [ ]:
label_list = ['B-O', 'B-AC', 'B-LF', 'I-LF']
print(label_list)

## Lemmatization

In [ ]:
nltk.download('wordnet')

In [ ]:
def combine_lists_elementwise(list_A, list_B):
  """
  Combines two 2D lists of strings element-wise into a 2D list of tuples.

  Args:
      list_A: A 2D list of strings (e.g., [['A', 'A', 'A'], ['A', 'A', 'A']]).
      list_B: Another 2D list of strings with the same dimensions as list_A.

  Returns:
      A 2D list of tuples, where each tuple combines corresponding elements from list_A and list_B.

  Raises:
      ValueError: If the dimensions of list_A and list_B don't match.
  """

  # Check if dimensions match
  if len(list_A) != len(list_B) or len(list_A[0]) != len(list_B[0]):
    raise ValueError("Dimensions of lists A and B must be equal.")

  # Create the resulting list using list comprehension
  return [[(a, b) for a, b in zip(row_a, row_b)] for row_a, row_b in zip(list_A, list_B)]

In [ ]:
def convert_pos_tag(nltk_tag):
    """
    Converts NLTK POS tags to the format expected by the lemmatizer.

    Args:
        nltk_tag: The POS tag in NLTK format (e.g., VBG, NNS).

    Returns:
        The corresponding POS tag for the lemmatizer (n, v, a, r, or s) or None if no match.
    """

    tag_map = {
        'NUM': '',  # Number (not handled by lemmatizer)
        'CCONJ': '',  # Coordinating conjunction (not handled)
        'PRON': '',  # Pronoun (not handled)
        'NOUN': 'n',   # Noun
        'SCONJ': '',  # Subordinating conjunction (not handled)
        'SYM': '',   # Symbol (not handled)
        'INTJ': '',  # Interjection (not handled)
        'ADJ': 'a',    # Adjective
        'ADP': '',   # Preposition (not handled)
        'PUNCT': '',  # Punctuation (not handled)
        'ADV': 'r',    # Adverb
        'AUX': 'v',    # Auxiliary verb
        'DET': '',   # Determiner (not handled)
        'VERB': 'v',   # Verb
        'X': '',      # Other (not handled)
        'PART': '',   # Particle (not handled)
        'PROPN': 'n',   # Proper noun
    }
    return tag_map.get(nltk_tag)

In [ ]:
def lemmatize_list(data, pos_tags):
    """
    Lemmatizes a 2D list of tokens using NLTK.

    Args:
        data: A 2D list of strings (tokens) to be lemmatized.

    Returns:
        A 2D list containing the lemmatized tokens.
    """

    # Initialize the WordNet lemmatizer
    lemmatizer = nltk.WordNetLemmatizer()

    pos_tags = [[convert_pos_tag(tag) for tag in row] for row in pos_tags]

    data = combine_lists_elementwise(data, pos_tags)


    # Lemmatize with part-of-speech information
    lemmatized_data = [[token if pos == '' else lemmatizer.lemmatize(token, pos) for token, pos in row] for row in data]

    return lemmatized_data

## Pre-Processing Pipeline

In [ ]:
def pre_process_data(tokens, pos_tags):
    # lemmatize the data
    data = lemmatize_list(tokens, pos_tags)
    # lowercase the data
    data = [[string.lower() for string in row] for row in data]
    return data

In [6]:
task = "ner"

# Model

# Pipeline

In [2]:
class MyPipeline(Pipeline):
    def _sanitize_parameters(self, **kwargs):
        preprocess_kwargs = {}
        if "maybe_arg" in kwargs:
            preprocess_kwargs["maybe_arg"] = kwargs["maybe_arg"]
        return preprocess_kwargs, {}, {}

    def preprocess(self, inputs, maybe_arg=2):
        model_input = Tensor(inputs["input_ids"])
        return {"model_input": model_input}

    def _forward(self, model_inputs):
        # model_inputs == {"model_input": model_input}
        outputs = self.model(**model_inputs)
        # Maybe {"logits": Tensor(...)}
        return outputs

    def postprocess(self, model_outputs):
        best_class = model_outputs["logits"].softmax(-1)
        return best_class

config.json:   0%|          | 0.00/688 [00:00<?, ?B/s]

C:\Users\Harry\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\huggingface_hub\file_download.py:149: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Harry\.cache\huggingface\hub\models--FacebookAI--roberta-large-mnli. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


model.safetensors:   0%|          | 0.00/1.43G [00:00<?, ?B/s]

Some weights of the model checkpoint at FacebookAI/roberta-large-mnli were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

[{'label': 'NEUTRAL', 'score': 0.7313137650489807}]

In [7]:
pipe = pipeline(task)
pipe("This is a test")

pipe("This is a test", top_k=2)

No model was supplied, defaulted to dbmdz/bert-large-cased-finetuned-conll03-english and revision f2482bf (https://huggingface.co/dbmdz/bert-large-cased-finetuned-conll03-english).
Using a pipeline without specifying a model name and revision in production is not recommended.


config.json:   0%|          | 0.00/998 [00:00<?, ?B/s]

C:\Users\Harry\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\huggingface_hub\file_download.py:149: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Harry\.cache\huggingface\hub\models--dbmdz--bert-large-cased-finetuned-conll03-english. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(mess

model.safetensors:   0%|          | 0.00/1.33G [00:00<?, ?B/s]

KeyboardInterrupt: 

# Server

In [ ]:
% python server.py